# **Pre-processing**

### **Import Libraries**

In [2]:
import pandas as pd
import re
import unicodedata
from underthesea import word_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

c:\Workplace\Data_Analyst\VNNewsClassifcation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
tqdm.pandas()

### **Load data**

In [31]:
df = pd.read_csv('../Data/vietnamese_news_cleaned_raw.csv')

In [32]:
df.head(2)

,title,content,category,raw_word_count
0,Nha Trang có thêm dự án nhà ở xã hội hơn 1.100...,UBND tỉnh Khánh Hòa vừa ban hành quyết định ch...,Bất động sản,530
1,Đề xuất Nhà nước thu hồi đất khi một nửa chủ s...,"Quan điểm này được ông Đỗ Đức Hồng Hà, Phó chủ...",Bất động sản,811


### **Chuẩn hóa Unicode**

In [33]:
def standardize_unicode(text):
    return unicodedata.normalize('NFC', text)

### **Làm sạch văn bản**

In [34]:
def clean_text_patterns(text):
    text = re.sub(r'<[^>]*>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'\S*@\S*\s?', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    return text

### **Chuẩn hóa định dạng văn bản**

In [35]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

### **Tách Token**

In [36]:
def tokenizer_pipeline(text):
    try:
        return word_tokenize(text, format='text')
    except Exception as e:
        print(f"Error tokenizing text: {e}")
        return ""
    

### **Loại bỏ stopword**

In [37]:
def load_stopwords(filepath):
    stopwords = set()
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    # Thêm biến thể với dấu gạch dưới
                    word_compatible = word.replace(' ', '_')
                    stopwords.add(word_compatible)
                    stopwords.add(word)
        return stopwords
    except Exception as e:
        print(f"Lỗi trong quá trình đọc file: {e}")
        return set()


In [38]:
stopword_file_path = '../vietnamese-stopwords.txt'
vn_stopwords = load_stopwords(stopword_file_path)

In [39]:
def remove_stopwords(text):
    if not text:
        return ""
    words = text.split()
    clean_words = [w for w in words if w not in vn_stopwords]
    return " ".join(clean_words)

### **Thực hiện pipeline Pre-processing**

In [40]:
def process_vietnamese_text(text):
    if not isinstance(text, str) or text is None:
        return ""
    text = standardize_unicode(text)
    text = clean_text_patterns(text)
    text = normalize_text(text)
    text = tokenizer_pipeline(text)
    text = remove_stopwords(text)
    return text

In [41]:
df['clean_content'] = df['content'].progress_apply(process_vietnamese_text)
df['clean_title'] = df['title'].progress_apply(process_vietnamese_text)

100%|██████████| 67159/67159 [01:33<00:00, 719.24it/s] 


In [42]:
sample = df[['content', 'clean_content']].sample(5)

for index, row in sample.iterrows():
    print("-" * 50)
    print(f"GỐC: {row['content'][:100]}...")
    print(f"XỬ LÝ: {row['clean_content'][:100]}...")

--------------------------------------------------
GỐC: 'Nhân viên khách sạn vô cảm' "Nhân viên khách sạn vô cảm, thiếu trách nhiệm, nhất là việc để khách n...
XỬ LÝ: nhân_viên khách_sạn vô_cảm nhân_viên khách_sạn vô_cảm trách_nhiệm nữ đi phòng h thể_hiện dịch_vụ khá...
--------------------------------------------------
GỐC: Gần bảy tháng sau khi tuyên bố đóng cửa các nhà hàng, quán cà phê và quán rượu do dịch COVID-19, Tổn...
XỬ LÝ: bảy tuyên_bố đóng_cửa nhà_hàng_quán cà_phê quán rượu dịch covid tổng_thống emmanuel macron thành_côn...
--------------------------------------------------
GỐC: Theo Rolling Stone, năm kênh MTV Music, MTV 80s, MTV 90s, Club MTV và MTV Live sẽ dừng phát sóng ở A...
XỬ LÝ: rolling stone kênh mtv music mtv s mtv s club mtv mtv_live dừng phát_sóng ireland australia lan_pháp...
--------------------------------------------------
GỐC: Bà Nguyễn Thu Hiền - Chủ tịch LĐLĐ huyện Lâm Bình cho biết, sau khi thẩm định điều kiện thực tế, trư...
XỬ LÝ: nguyễn_thu_hiền chủ_

In [43]:
df.head(5)

,title,content,category,raw_word_count,clean_content,clean_title
0,Nha Trang có thêm dự án nhà ở xã hội hơn 1.100...,UBND tỉnh Khánh Hòa vừa ban hành quyết định ch...,Bất động sản,530,ubnd tỉnh khánh_hòa ban_hành quyết_định chấp_t...,nha_trang dự_án xã_hội tỷ đồng
1,Đề xuất Nhà nước thu hồi đất khi một nửa chủ s...,"Quan điểm này được ông Đỗ Đức Hồng Hà, Phó chủ...",Bất động sản,811,quan_điểm đỗ đức hồng_hà phó chủ_nhiệm ủy_ban ...,đề_xuất nhà_nước thu_hồi đất một_nửa chủ_sở_hữ...
2,Xu hướng dùng nội thất inox trong thiết kế khô...,"Với đặc tính chống gỉ, chịu lực, dễ vệ sinh và...",Bất động sản,911,đặc_tính chống gỉ lực vệ_sinh bề_mặt thẩm_mỹ c...,xu_hướng nội thất_inox thiết_kế không_gian hiệ...
3,Cải tạo căn hộ 48 m2 cho 3 người ở với 280 tri...,"Căn hộ có diện tích 48 m2, tọa lạc tại một tòa...",Bất động sản,869,căn_hộ diện_tích m tọa_lạc tòa chung_cư tp hcm...,cải_tạo căn_hộ m triệu đồng
4,Thu ngân sách TP HCM vượt cùng kỳ nhờ đất đai,Thông tin được Phó giám đốc Sở Tài chính TP HC...,Bất động sản,603,thông_tin phó giám_đốc sở tài_chính tp hcm hoà...,thu ngân_sách tp hcm kỳ đất_đai


### **Kiểm tra có giá trị nào "" hoặc NaN sau khi pre-processing**

In [ ]:
df.loc[(df['clean_title'] == "") | (df['clean_content'] == "")].head()

In [54]:
df = df[(df['clean_title'] != "") & (df['clean_content'] != "")]

In [55]:
df.loc[(df['clean_title'] == "") | (df['clean_content'] == "")].head()

,title,content,category,raw_word_count,clean_content,clean_title


### **Lưu data đã pre-processing**

In [56]:
df.to_csv('../Data/vietnamese_news_preprocessed.csv', index=False)